# 自己的情感分析任务
针对自己的数据集的情感分析任务，加载之前训练好的预训练模型，用自己的游记文本数据再次训练模型，学习游记文本中的语义


# 预处理自己的游记文本数据集
对标注好的游记文本数据进行打乱处理，划分好训练集、验证集和测试集

In [1]:
import torch
import torch.utils.data as D
import pandas as pd
from datasets import load_dataset

In [3]:
help(load_dataset)

Help on function load_dataset in module datasets.load:

load_dataset(path: str, name: Union[str, NoneType] = None, data_dir: Union[str, NoneType] = None, data_files: Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]], NoneType] = None, split: Union[str, datasets.splits.Split, NoneType] = None, cache_dir: Union[str, NoneType] = None, features: Union[datasets.features.features.Features, NoneType] = None, download_config: Union[datasets.download.download_config.DownloadConfig, NoneType] = None, download_mode: Union[datasets.download.download_manager.DownloadMode, NoneType] = None, ignore_verifications: bool = False, keep_in_memory: Union[bool, NoneType] = None, save_infos: bool = False, revision: Union[str, datasets.utils.version.Version, NoneType] = None, use_auth_token: Union[bool, str, NoneType] = None, task: Union[str, datasets.tasks.base.TaskTemplate, NoneType] = None, streaming: bool = False, **config_kwargs) -> Union[datasets.dataset_dict.DatasetDict, datasets.arrow_

In [6]:
dataset = load_dataset('csv',data_files='../data/MyDataset/dataset.csv')

Using custom data configuration default-4d90218a9b3a1753


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/chenli/.cache/huggingface/datasets/csv/default-4d90218a9b3a1753/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
dataset['train']

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2445
    })
})

In [9]:
shuffle_dataset = dataset.shuffle(9) # 将数据打乱，数值越大，混乱程度越大

In [12]:
train_size = int(0.8 * len(shuffle_dataset))
valid_size = int(0.1 * len(shuffle_dataset))
test_size = len(shuffle_dataset) - train_size - valid_size
train_dataset,valid_dataset,test_dataset = torch.utils.data.random_split(shuffle_dataset, [train_size,valid_size,test_size])

In [13]:
print(train_size)
print(valid_size)
print(test_size)

0
0
1
